In [1]:
import torchvision.transforms as transforms
import torch
import torch.utils.data as data
import numpy as np
from sklearn import svm
import librosa
import csv
import time

from trainDataset import TrainDataset
from testDataset import TestDataset

toFloat = transforms.Lambda(lambda x: x / np.iinfo(np.int16).max)

trainDataset = TrainDataset("kaggle-train", transform=toFloat)
print(len(trainDataset))

testDataset = TestDataset("kaggle-test", transform=toFloat)
print(len(testDataset))

/home/amarafioti/.local/lib/python3.5/site-packages/numba/errors.py:131: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


269776
4096


In [2]:
familyClassifier = svm.SVC()

trainLoader = data.DataLoader(trainDataset, batch_size=6400, shuffle=True)
start = time.time()
for samples, instrumentsFamily in trainLoader:
    familyClassifier.fit(np.array(samples.data)[:, :16000], np.array(instrumentsFamily.data))
    break # SVM is only fitted to a fixed size of data
print("Fitting the SVM took " + str((time.time()-start)/60) + "mins")

Fitting the SVM took 18.701761484146118mins


In [3]:
batch_size = 32
testloader = data.DataLoader(testDataset, batch_size=batch_size, shuffle=False) #!!! Shuffle should be false

familyPredictions = np.zeros(len(testDataset), dtype=np.int)
start = time.time()
for index, samples in enumerate(testloader):
    familyPredictions[index*batch_size:(index+1)*batch_size] = familyClassifier.predict(np.array(samples.data)[:, :16000])
print("Classifying took " + str((time.time()-start)/60) + "mins")

Classifying took 7.246222802003225mins


In [4]:
familyPredictionStrings = trainDataset.transformInstrumentsFamilyToString(familyPredictions.astype(int))

with open('SVM-time-submission.csv', 'w', newline='') as writeFile:
    fieldnames = ['Id', 'Expected']
    writer = csv.DictWriter(writeFile, fieldnames=fieldnames, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writeheader()
    for index in range(len(testDataset)):
        writer.writerow({'Id': index, 'Expected': familyPredictionStrings[index]})


/home/amarafioti/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Now let's see what we can do with a more informed SVM

In [5]:
sample_rate = 16000

informedFamilyClassifier = svm.SVC()

def computeMelspectrogram(numpyArray, sample_rate):
    S = librosa.feature.melspectrogram(y=numpyArray, sr=sample_rate, n_mels=128, fmax=8000)
    return np.log(S+1e-4)

trainLoader = data.DataLoader(trainDataset, batch_size=6400, shuffle=True)
start = time.time()
for samples, instrumentsFamily in trainLoader:
    mfccs = np.zeros((len(samples), 128))
    for index, sample in enumerate(samples.data):
        mfccs[index] = np.mean(computeMelspectrogram(sample.numpy(), sample_rate), axis=1)

    informedFamilyClassifier.fit(mfccs, instrumentsFamily.numpy())
    break # SVM is only fitted to a fixed size of data
print("Fitting the SVM took " + str((time.time()-start)/60) + "mins")

Fitting the SVM took 1.5752114375432333mins


In [6]:
batch_size = 32
testloader = data.DataLoader(testDataset, batch_size=batch_size, shuffle=False) #!!! Shuffle should be false

informedFamilyPredictions = np.zeros(len(testDataset), dtype=np.int)
start = time.time()
for index, samples in enumerate(testloader):
    mfccs = np.zeros((len(samples), 128))
    for inner_index, sample in enumerate(samples.data):
        mfccs[inner_index] = np.mean(computeMelspectrogram(sample.numpy(), sample_rate), axis=1)

    informedFamilyPredictions[index*batch_size:(index+1)*batch_size] = informedFamilyClassifier.predict(mfccs)
print("Classifying took " + str((time.time()-start)/60) + "mins")

Classifying took 0.737498418490092mins


In [7]:
informedFamilyPredictionStrings = trainDataset.transformInstrumentsFamilyToString(informedFamilyPredictions.astype(int))

with open('SVM-informed-submission.csv', 'w', newline='') as writeFile:
    fieldnames = ['Id', 'Expected']
    writer = csv.DictWriter(writeFile, fieldnames=fieldnames, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writeheader()
    for index in range(len(testDataset)):
        writer.writerow({'Id': index, 'Expected': informedFamilyPredictionStrings[index]})


/home/amarafioti/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
import matplotlib.pyplot as plt

for i in range(32):
    plt.plot(mfccs[i])
    plt.show()